<a href="https://colab.research.google.com/github/Du-nara/ME421-Mechanical-Systems-Lab-A3/blob/main/Vibration/E_20_260_Vibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WEEK #1**





# Activity 1: Derive the 2-DOF Model for TecQuipment Vibration Analyzer

## System Description

We consider a **rigid beam** vibration system with the following components:

### Physical Components:
1. **Rigid Beam**: Pinned at one end (Point O), with length $L$, mass $m_b$, moment of inertia $J_b$
2. **Motor**: Mounted on the beam at distance $L_m$ from the pivot, with mass $m_{motor}$
3. **Unbalanced Mass**: Mass $m_u$ rotating at eccentricity $e$ with angular velocity $\omega$
4. **Spring with Mass**: The spring has mass $m_s$, stiffness $k$, attached to beam at distance $L_s$ from pivot
5. **Damper**: Viscous damping coefficient $c$

### System Configuration:
- One end of the beam is **pinned** (pivot point O) - allows rotation only
- The other end has a **spring** connecting it to the frame
- The spring itself has mass $m_s$ that can move independently
- An **unbalanced motor** creates harmonic excitation

### Degrees of Freedom:

**DOF 1: $\theta(t)$** - Angular displacement of rigid beam about pinned end [rad]

**DOF 2: $x_s(t)$** - Displacement of spring mass from equilibrium position [m]

The spring creates coupling between the beam rotation and the spring mass motion, resulting in a **2-DOF system**.

---

## Step 1: Define Coordinate System and Assumptions

### Coordinate Definition:
- Origin at pivot point O
- $\theta$: measured positive counterclockwise
- $x_s$: measured positive in the direction away from beam

### Key Assumptions:
1. **Small angle approximation**: $\sin\theta \approx \theta$, $\cos\theta \approx 1 - \frac{\theta^2}{2}$
2. **Rigid beam**: No flexibility or bending
3. **Linear spring**: Follows Hooke's law $F = k\delta$
4. **Viscous damping**: $F_d = c\dot{\delta}$

---

## Step 2: Kinetic Energy Derivation

The total kinetic energy consists of three parts:

### 2.1 Beam and Motor Assembly (Rigid Body Rotation)

The beam and motor rotate together about the pivot. The total moment of inertia about pivot O is:

$$J_{total} = J_b + m_b \bar{r}^2 + m_{motor} L_m^2$$

where $\bar{r}$ is the distance from pivot to beam's center of mass.

Rotational kinetic energy:
$$T_{beam} = \frac{1}{2} J_{total} \dot{\theta}^2$$

### 2.2 Spring Mass (Translation)

The spring mass moves with velocity $\dot{x}_s$:

$$T_{spring} = \frac{1}{2} m_s \dot{x}_s^2$$

### 2.3 Unbalanced Mass (Complex Motion)

The unbalanced mass has position (in Cartesian coordinates):

**Horizontal position:**
$$x_u = L_m \sin\theta + e\cos(\omega t)$$

For small $\theta$:
$$x_u \approx L_m \theta + e\cos(\omega t)$$

**Vertical position:**
$$y_u = -L_m \cos\theta + e\sin(\omega t)$$

For small $\theta$:
$$y_u \approx -L_m + e\sin(\omega t)$$

**Velocities:**
$$\dot{x}_u = L_m \dot{\theta} - e\omega\sin(\omega t)$$
$$\dot{y}_u = e\omega\cos(\omega t)$$

**Kinetic energy:**
$$T_{unbalance} = \frac{1}{2} m_u [(\dot{x}_u)^2 + (\dot{y}_u)^2]$$

$$T_{unbalance} = \frac{1}{2} m_u [(L_m \dot{\theta} - e\omega\sin(\omega t))^2 + (e\omega\cos(\omega t))^2]$$

Expanding:
$$T_{unbalance} = \frac{1}{2} m_u [L_m^2 \dot{\theta}^2 - 2L_m e\omega\dot{\theta}\sin(\omega t) + e^2\omega^2]$$

### Total Kinetic Energy:

$$T = T_{beam} + T_{spring} + T_{unbalance}$$

$$\boxed{T = \frac{1}{2}(J_{total} + m_u L_m^2)\dot{\theta}^2 + \frac{1}{2}m_s\dot{x}_s^2 - m_u L_m e\omega\dot{\theta}\sin(\omega t) + \frac{1}{2}m_u e^2\omega^2}$$

For convenience, define the effective moment of inertia:
$$J_{eff} = J_{total} + m_u L_m^2 = J_b + m_b\bar{r}^2 + (m_{motor} + m_u)L_m^2$$

---

## Step 3: Potential Energy Derivation

The potential energy has two components: elastic and gravitational.

### 3.1 Spring Elastic Potential Energy

The spring connects the beam at position $L_s$ to the spring mass. When the beam rotates by angle $\theta$, the beam attachment point moves vertically by:
$$y_{beam} = L_s \theta$$ (for small angles)

The spring mass is at position $x_s$. The spring deformation is:
$$\delta = x_s - L_s \theta$$

Elastic potential energy:
$$V_{spring} = \frac{1}{2} k (x_s - L_s \theta)^2$$

### 3.2 Gravitational Potential Energy

Taking the pivot as reference height (h = 0):

**Beam center of mass:**
$$h_{beam} = -\bar{r}\cos\theta \approx -\bar{r}(1 - \frac{\theta^2}{2}) = -\bar{r} + \frac{1}{2}\bar{r}\theta^2$$

$$V_{beam} = m_b g h_{beam} = -m_b g \bar{r} + \frac{1}{2}m_b g \bar{r}\theta^2$$

**Motor:**
$$h_{motor} = -L_m\cos\theta \approx -L_m + \frac{1}{2}L_m\theta^2$$

$$V_{motor} = m_{motor} g h_{motor} = -m_{motor} g L_m + \frac{1}{2}m_{motor} g L_m\theta^2$$

**Spring mass:**
$$V_{spring,mass} = -m_s g x_s$$

### Total Potential Energy:

$$V = V_{spring} + V_{beam} + V_{motor} + V_{spring,mass}$$

$$V = \frac{1}{2}k(x_s - L_s\theta)^2 + \frac{1}{2}(m_b g \bar{r} + m_{motor} g L_m)\theta^2 - m_s g x_s + const$$

Define the **gravity stiffness term**:
$$k_g = m_b g \bar{r} + m_{motor} g L_m$$

$$\boxed{V = \frac{1}{2}k(x_s - L_s\theta)^2 + \frac{1}{2}k_g\theta^2 - m_s g x_s}$$ (ignoring constants)

---

## Step 4: Dissipation Function

The viscous damper acts on the relative velocity between the spring mass and the beam attachment point:

**Relative velocity:**
$$v_{rel} = \dot{x}_s - L_s\dot{\theta}$$

**Rayleigh dissipation function:**
$$\boxed{\mathcal{D} = \frac{1}{2}c(\dot{x}_s - L_s\dot{\theta})^2}$$

---

## Step 5: Apply Lagrange's Equations

The Lagrange's equations with dissipation are:

$$\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}_i}\right) - \frac{\partial T}{\partial q_i} + \frac{\partial V}{\partial q_i} + \frac{\partial \mathcal{D}}{\partial \dot{q}_i} = Q_i$$

where $Q_i$ are the generalized forces.

### 5.1 Equation for $\theta$:

**Kinetic energy terms:**
$$\frac{\partial T}{\partial \dot{\theta}} = J_{eff}\dot{\theta} - m_u L_m e\omega\sin(\omega t)$$

$$\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{\theta}}\right) = J_{eff}\ddot{\theta} - m_u L_m e\omega^2\cos(\omega t)$$

$$\frac{\partial T}{\partial \theta} = 0$$

**Potential energy term:**
$$\frac{\partial V}{\partial \theta} = k(x_s - L_s\theta)(-L_s) + k_g\theta = -kL_s x_s + kL_s^2\theta + k_g\theta$$

**Dissipation term:**
$$\frac{\partial \mathcal{D}}{\partial \dot{\theta}} = c(\dot{x}_s - L_s\dot{\theta})(-L_s) = -cL_s\dot{x}_s + cL_s^2\dot{\theta}$$

**Generalized force** (from unbalanced mass):
$$Q_\theta = m_u L_m e\omega^2\cos(\omega t)$$

**Combining all terms:**
$$J_{eff}\ddot{\theta} - m_u L_m e\omega^2\cos(\omega t) - 0 + (-kL_s x_s + kL_s^2\theta + k_g\theta) + (-cL_s\dot{x}_s + cL_s^2\dot{\theta}) = m_u L_m e\omega^2\cos(\omega t)$$

Simplifying:
$$\boxed{J_{eff}\ddot{\theta} + cL_s^2\dot{\theta} - cL_s\dot{x}_s + (kL_s^2 + k_g)\theta - kL_s x_s = 0}$$

### 5.2 Equation for $x_s$:

**Kinetic energy terms:**
$$\frac{\partial T}{\partial \dot{x}_s} = m_s\dot{x}_s$$

$$\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{x}_s}\right) = m_s\ddot{x}_s$$

$$\frac{\partial T}{\partial x_s} = 0$$

**Potential energy term:**
$$\frac{\partial V}{\partial x_s} = k(x_s - L_s\theta)(1) - m_s g = kx_s - kL_s\theta - m_s g$$

**Dissipation term:**
$$\frac{\partial \mathcal{D}}{\partial \dot{x}_s} = c(\dot{x}_s - L_s\dot{\theta})(1) = c\dot{x}_s - cL_s\dot{\theta}$$

**Generalized force:**
$$Q_{x_s} = 0$$

**Combining all terms:**
$$m_s\ddot{x}_s - 0 + (kx_s - kL_s\theta - m_s g) + (c\dot{x}_s - cL_s\dot{\theta}) = 0$$

$$\boxed{m_s\ddot{x}_s + c\dot{x}_s - cL_s\dot{\theta} + kx_s - kL_s\theta = m_s g}$$

---

## Step 6: Final Equations in Matrix Form

Rearranging the two equations:

**Equation 1:**
$$J_{eff}\ddot{\theta} + cL_s^2\dot{\theta} - cL_s\dot{x}_s + (kL_s^2 + k_g)\theta - kL_s x_s = 0$$

**Equation 2:**
$$m_s\ddot{x}_s - cL_s\dot{\theta} + c\dot{x}_s - kL_s\theta + kx_s = m_s g$$

### Matrix Form:

$$\mathbf{M}\ddot{\mathbf{q}} + \mathbf{C}\dot{\mathbf{q}} + \mathbf{K}\mathbf{q} = \mathbf{F}(t)$$

where:

$$\mathbf{q} = \begin{bmatrix} \theta \\ x_s \end{bmatrix}$$

### **Mass Matrix:**
$$\mathbf{M} = \begin{bmatrix} J_{eff} & 0 \\ 0 & m_s \end{bmatrix}$$

### **Damping Matrix:**
$$\mathbf{C} = \begin{bmatrix} cL_s^2 & -cL_s \\ -cL_s & c \end{bmatrix}$$

### **Stiffness Matrix:**
$$\mathbf{K} = \begin{bmatrix} kL_s^2 + k_g & -kL_s \\ -kL_s & k \end{bmatrix}$$

### **Force Vector:**
$$\mathbf{F}(t) = \begin{bmatrix} 0 \\ m_s g \end{bmatrix}$$

Note: The harmonic excitation from the unbalanced motor can be written separately or the equations can be modified for forced vibration analysis.

---

## Final Result: Complete 2-DOF Equations of Motion

$$\boxed{
\begin{bmatrix}
J_{eff} & 0 \\
0 & m_s
\end{bmatrix}
\begin{bmatrix}
\ddot{\theta} \\
\ddot{x}_s
\end{bmatrix} +
\begin{bmatrix}
cL_s^2 & -cL_s \\
-cL_s & c
\end{bmatrix}
\begin{bmatrix}
\dot{\theta} \\
\dot{x}_s
\end{bmatrix} +
\begin{bmatrix}
kL_s^2 + k_g & -kL_s \\
-kL_s & k
\end{bmatrix}
\begin{bmatrix}
\theta \\
x_s
\end{bmatrix} =
\begin{bmatrix}
0 \\
m_s g
\end{bmatrix}
}$$

where:
- $J_{eff} = J_b + m_b\bar{r}^2 + (m_{motor} + m_u)L_m^2$ = Total effective moment of inertia
- $k_g = m_b g \bar{r} + m_{motor} g L_m$ = Gravity stiffness term

### Physical Interpretation:

1. **Coupling**: The off-diagonal terms $-cL_s$ and $-kL_s$ indicate that the two DOFs are **coupled** through the spring-damper connection.

2. **Mass matrix diagonal**: No inertial coupling - the rotational inertia and translational mass are independent.

3. **Constant forcing**: The $m_s g$ term represents the static weight of the spring mass, which establishes the equilibrium position.

4. **Harmonic excitation**: For forced vibration analysis with the unbalanced motor, we can add harmonic forcing to the first equation or solve about the equilibrium and add the motor excitation term.

### For Forced Vibration Analysis:

The equations can be modified to include the motor excitation explicitly by measuring displacements from equilibrium and adding the excitation term:

$$\mathbf{M}\ddot{\mathbf{q}} + \mathbf{C}\dot{\mathbf{q}} + \mathbf{K}\mathbf{q} = \begin{bmatrix} m_u e \omega^2 L_m \cos(\omega t) \\ 0 \end{bmatrix}$$

where the coordinates are measured from the static equilibrium position.

---




# Activity 2: Estimate Damping Ratios of the First Two Dominant Modes

## Overview

From Activity 1, we have the 2-DOF equations of motion:

$$\mathbf{M}\ddot{\mathbf{q}} + \mathbf{C}\dot{\mathbf{q}} + \mathbf{K}\mathbf{q} = \mathbf{F}(t)$$

To estimate the damping ratios, we need to:
1. Find the **natural frequencies** $\omega_{n1}, \omega_{n2}$ (undamped eigenvalue problem)
2. Find the **mode shapes** $\boldsymbol{\phi}_1, \boldsymbol{\phi}_2$ (eigenvectors)
3. Calculate the **modal damping ratios** $\zeta_1, \zeta_2$ using modal transformation

---

## Step 1: Undamped Natural Frequencies

For the undamped system, we set $\mathbf{C} = 0$ and $\mathbf{F} = 0$:

$$\mathbf{M}\ddot{\mathbf{q}} + \mathbf{K}\mathbf{q} = 0$$

Assuming harmonic solution: $\mathbf{q}(t) = \boldsymbol{\phi}e^{i\omega t}$

Substituting:
$$(-\omega^2\mathbf{M} + \mathbf{K})\boldsymbol{\phi} = 0$$

or equivalently:
$$(\mathbf{K} - \omega^2\mathbf{M})\boldsymbol{\phi} = 0$$

This is the **eigenvalue problem**. Non-trivial solutions exist when:

$$\det(\mathbf{K} - \omega^2\mathbf{M}) = 0$$

### Substitute the Matrices:

From Activity 1:

$$\mathbf{M} = \begin{bmatrix} J_{eff} & 0 \\ 0 & m_s \end{bmatrix}$$

$$\mathbf{K} = \begin{bmatrix} kL_s^2 + k_g & -kL_s \\ -kL_s & k \end{bmatrix}$$

### Form the Characteristic Equation:

$$\det\begin{bmatrix} kL_s^2 + k_g - \omega^2 J_{eff} & -kL_s \\ -kL_s & k - \omega^2 m_s \end{bmatrix} = 0$$

Expanding the determinant:
$$(kL_s^2 + k_g - \omega^2 J_{eff})(k - \omega^2 m_s) - (-kL_s)(-kL_s) = 0$$

$$(kL_s^2 + k_g - \omega^2 J_{eff})(k - \omega^2 m_s) - k^2L_s^2 = 0$$

Expanding:
$$(kL_s^2 + k_g)k - (kL_s^2 + k_g)\omega^2 m_s - \omega^2 J_{eff} k + \omega^4 J_{eff} m_s - k^2L_s^2 = 0$$

Rearranging:
$$J_{eff} m_s \omega^4 - [(kL_s^2 + k_g)m_s + kJ_{eff}]\omega^2 + [k(kL_s^2 + k_g) - k^2L_s^2] = 0$$

Simplifying the constant term:
$$k(kL_s^2 + k_g) - k^2L_s^2 = k^2L_s^2 + kk_g - k^2L_s^2 = kk_g$$

### Characteristic Equation (Quadratic in $\omega^2$):

$$\boxed{J_{eff} m_s \omega^4 - [(kL_s^2 + k_g)m_s + kJ_{eff}]\omega^2 + kk_g = 0}$$

Let $\lambda = \omega^2$. Then:
$$a\lambda^2 + b\lambda + c = 0$$

where:
- $a = J_{eff} m_s$
- $b = -[(kL_s^2 + k_g)m_s + kJ_{eff}]$
- $c = kk_g$

### Solution Using Quadratic Formula:

$$\lambda_{1,2} = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$$

$$\omega^2_{1,2} = \frac{[(kL_s^2 + k_g)m_s + kJ_{eff}] \pm \sqrt{[(kL_s^2 + k_g)m_s + kJ_{eff}]^2 - 4J_{eff}m_s \cdot kk_g}}{2J_{eff}m_s}$$

### Natural Frequencies:

$$\boxed{\omega_{n1} = \sqrt{\lambda_1}, \quad \omega_{n2} = \sqrt{\lambda_2}}$$

where $\omega_{n1} < \omega_{n2}$ (first mode is lower frequency)

### Alternative Compact Form:

Define:
- $\Omega_1^2 = \frac{kL_s^2 + k_g}{J_{eff}}$ (uncoupled beam natural frequency squared)
- $\Omega_2^2 = \frac{k}{m_s}$ (uncoupled spring natural frequency squared)

Then:
$$\omega_{n1,2}^2 = \frac{1}{2}\left[(\Omega_1^2 + \Omega_2^2) \mp \sqrt{(\Omega_1^2 - \Omega_2^2)^2 + 4\frac{k^2L_s^2}{J_{eff}m_s}}\right]$$

---

## Step 2: Mode Shapes

For each natural frequency $\omega_{ni}$, we find the corresponding mode shape $\boldsymbol{\phi}_i$ from:

$$(\mathbf{K} - \omega_{ni}^2\mathbf{M})\boldsymbol{\phi}_i = 0$$

Let $\boldsymbol{\phi}_i = \begin{bmatrix} \phi_{1i} \\ \phi_{2i} \end{bmatrix}$ where:
- $\phi_{1i}$ = amplitude of $\theta$ in mode $i$
- $\phi_{2i}$ = amplitude of $x_s$ in mode $i$

### For Mode 1 ($\omega_{n1}$):

$$\begin{bmatrix} kL_s^2 + k_g - \omega_{n1}^2 J_{eff} & -kL_s \\ -kL_s & k - \omega_{n1}^2 m_s \end{bmatrix}\begin{bmatrix} \phi_{11} \\ \phi_{21} \end{bmatrix} = 0$$

From the first equation:
$$(kL_s^2 + k_g - \omega_{n1}^2 J_{eff})\phi_{11} - kL_s\phi_{21} = 0$$

Solving for the ratio:
$$r_1 = \frac{\phi_{21}}{\phi_{11}} = \frac{kL_s^2 + k_g - \omega_{n1}^2 J_{eff}}{kL_s}$$

Normalizing so that $\phi_{11} = 1$:
$$\boxed{\boldsymbol{\phi}_1 = \begin{bmatrix} 1 \\ r_1 \end{bmatrix} = \begin{bmatrix} 1 \\ \frac{kL_s^2 + k_g - \omega_{n1}^2 J_{eff}}{kL_s} \end{bmatrix}}$$

### For Mode 2 ($\omega_{n2}$):

Similarly:
$$r_2 = \frac{\phi_{22}}{\phi_{12}} = \frac{kL_s^2 + k_g - \omega_{n2}^2 J_{eff}}{kL_s}$$

$$\boxed{\boldsymbol{\phi}_2 = \begin{bmatrix} 1 \\ r_2 \end{bmatrix} = \begin{bmatrix} 1 \\ \frac{kL_s^2 + k_g - \omega_{n2}^2 J_{eff}}{kL_s} \end{bmatrix}}$$

### Alternative Form:

We can also express using the second row:
$$r_i = \frac{kL_s}{k - \omega_{ni}^2 m_s}$$

---

## Step 3: Modal Damping Ratios

To find the damping ratios for each mode, we use **modal transformation**.

### Modal Matrix:

$$\boldsymbol{\Phi} = [\boldsymbol{\phi}_1 \quad \boldsymbol{\phi}_2] = \begin{bmatrix} 1 & 1 \\ r_1 & r_2 \end{bmatrix}$$

### Modal Transformation:

Transform the physical coordinates to modal coordinates:
$$\mathbf{q} = \boldsymbol{\Phi}\mathbf{p}$$

where $\mathbf{p} = \begin{bmatrix} p_1 \\ p_2 \end{bmatrix}$ are the modal coordinates.

Substituting into the equation of motion and pre-multiplying by $\boldsymbol{\Phi}^T$:

$$\boldsymbol{\Phi}^T\mathbf{M}\boldsymbol{\Phi}\ddot{\mathbf{p}} + \boldsymbol{\Phi}^T\mathbf{C}\boldsymbol{\Phi}\dot{\mathbf{p}} + \boldsymbol{\Phi}^T\mathbf{K}\boldsymbol{\Phi}\mathbf{p} = \boldsymbol{\Phi}^T\mathbf{F}$$

### Modal Mass Matrix:

$$\mathbf{M}_{modal} = \boldsymbol{\Phi}^T\mathbf{M}\boldsymbol{\Phi} = \begin{bmatrix} m_1 & 0 \\ 0 & m_2 \end{bmatrix}$$

where:
$$m_1 = \boldsymbol{\phi}_1^T\mathbf{M}\boldsymbol{\phi}_1 = J_{eff} + m_s r_1^2$$
$$m_2 = \boldsymbol{\phi}_2^T\mathbf{M}\boldsymbol{\phi}_2 = J_{eff} + m_s r_2^2$$

### Modal Stiffness Matrix:

$$\mathbf{K}_{modal} = \boldsymbol{\Phi}^T\mathbf{K}\boldsymbol{\Phi} = \begin{bmatrix} k_1 & 0 \\ 0 & k_2 \end{bmatrix}$$

where:
$$k_1 = m_1\omega_{n1}^2, \quad k_2 = m_2\omega_{n2}^2$$

### Modal Damping Matrix:

$$\mathbf{C}_{modal} = \boldsymbol{\Phi}^T\mathbf{C}\boldsymbol{\Phi}$$

Calculating each element:

$$\mathbf{C}_{modal} = \boldsymbol{\Phi}^T \begin{bmatrix} cL_s^2 & -cL_s \\ -cL_s & c \end{bmatrix} \boldsymbol{\Phi}$$

$$\boldsymbol{\Phi}^T\mathbf{C} = \begin{bmatrix} 1 & r_1 \\ 1 & r_2 \end{bmatrix}\begin{bmatrix} cL_s^2 & -cL_s \\ -cL_s & c \end{bmatrix} = \begin{bmatrix} cL_s^2 - r_1 cL_s & -cL_s + r_1 c \\ cL_s^2 - r_2 cL_s & -cL_s + r_2 c \end{bmatrix}$$

$$C_{11} = (cL_s^2 - r_1 cL_s)(1) + (-cL_s + r_1 c)(r_1) = cL_s^2 - r_1 cL_s - r_1 cL_s + r_1^2 c$$

$$\boxed{C_{11} = c(L_s^2 - 2r_1 L_s + r_1^2) = c(L_s - r_1)^2}$$

Similarly:
$$\boxed{C_{22} = c(L_s - r_2)^2}$$

The off-diagonal terms $C_{12} = C_{21} \neq 0$ in general (damping may not be perfectly proportional).

### Modal Damping Ratios:

For each mode, the damping ratio is:

$$\zeta_i = \frac{C_{ii}}{2\sqrt{k_i m_i}} = \frac{C_{ii}}{2m_i\omega_{ni}}$$

$$\boxed{\zeta_1 = \frac{c(L_s - r_1)^2}{2(J_{eff} + m_s r_1^2)\omega_{n1}}}$$

$$\boxed{\zeta_2 = \frac{c(L_s - r_2)^2}{2(J_{eff} + m_s r_2^2)\omega_{n2}}}$$

---

## Step 4: Summary of Procedure

### To Estimate Damping Ratios:

1. **Calculate natural frequencies** $\omega_{n1}, \omega_{n2}$ from the characteristic equation

2. **Calculate mode shape ratios** $r_1, r_2$:
   $$r_i = \frac{kL_s^2 + k_g - \omega_{ni}^2 J_{eff}}{kL_s}$$

3. **Calculate modal masses**:
   $$m_i = J_{eff} + m_s r_i^2$$

4. **Calculate modal damping coefficients**:
   $$C_{ii} = c(L_s - r_i)^2$$

5. **Calculate damping ratios**:
   $$\zeta_i = \frac{C_{ii}}{2m_i\omega_{ni}}$$

---

## Step 5: Physical Interpretation

### Mode 1 (Lower Frequency):
- Typically corresponds to **in-phase motion** where beam and spring move together
- Lower frequency = softer effective spring
- Damping ratio depends on relative motion between beam and spring

### Mode 2 (Higher Frequency):
- Typically corresponds to **out-of-phase motion** where beam and spring move oppositely
- Higher frequency = stiffer effective spring
- Usually has different damping characteristics than Mode 1

### Damping Characteristics:
- If $r_i \approx L_s$: Little relative motion → **low damping** for that mode
- If $r_i$ very different from $L_s$: Large relative motion → **high damping** for that mode
- The term $(L_s - r_i)^2$ represents the squared relative amplitude

---

## Step 6: Special Cases

### Case 1: Proportional Damping
If $\mathbf{C} = \alpha\mathbf{M} + \beta\mathbf{K}$ (Rayleigh damping), then:
$$\zeta_i = \frac{\alpha}{2\omega_{ni}} + \frac{\beta\omega_{ni}}{2}$$

### Case 2: Light Damping
If damping is light ($\zeta_i \ll 1$), natural frequencies are approximately:
$$\omega_{di} \approx \omega_{ni}\sqrt{1-\zeta_i^2} \approx \omega_{ni}$$

### Case 3: Uncoupled Limits
If spring mass is negligible ($m_s \to 0$):
$$\omega_{n1} \to 0, \quad \omega_{n2} \to \sqrt{\frac{kL_s^2 + k_g}{J_{eff}}}$$

---

## Experimental Estimation Methods

In practice, damping ratios can be estimated from experimental data using:

### Method 1: Logarithmic Decrement
From free vibration response:
$$\zeta_i = \frac{1}{2\pi n}\ln\left(\frac{x_0}{x_n}\right)$$
where $x_0$ and $x_n$ are amplitudes $n$ cycles apart.

### Method 2: Half-Power Bandwidth
From frequency response function:
$$\zeta_i = \frac{\omega_2 - \omega_1}{2\omega_{ni}}$$
where $\omega_1, \omega_2$ are frequencies at amplitude $= \frac{1}{\sqrt{2}}$ of peak.

### Method 3: Peak Amplitude
From FRF peak:
$$\zeta_i \approx \frac{1}{2Q} = \frac{1}{2}\frac{X(\omega_{ni})\omega_{ni}^2}{F_0}$$

---

## Final Formulas for Implementation

### Natural Frequencies:
$$\omega_{n1,2}^2 = \frac{(kL_s^2 + k_g)m_s + kJ_{eff} \mp \sqrt{[(kL_s^2 + k_g)m_s + kJ_{eff}]^2 - 4J_{eff}m_s kk_g}}{2J_{eff}m_s}$$

### Mode Shape Ratios:
$$r_1 = \frac{kL_s^2 + k_g - \omega_{n1}^2 J_{eff}}{kL_s}, \quad r_2 = \frac{kL_s^2 + k_g - \omega_{n2}^2 J_{eff}}{kL_s}$$

### Damping Ratios:
$$\zeta_1 = \frac{c(L_s - r_1)^2}{2(J_{eff} + m_s r_1^2)\omega_{n1}}, \quad \zeta_2 = \frac{c(L_s - r_2)^2}{2(J_{eff} + m_s r_2^2)\omega_{n2}}$$

where:
- $J_{eff} = J_b + m_b\bar{r}^2 + (m_{motor} + m_u)L_m^2$
- $k_g = m_b g \bar{r} + m_{motor} g L_m$

---

## Activity 2 Complete! ✅

We have derived:
✅ Natural frequencies of both modes
✅ Mode shapes for both modes
✅ Modal damping ratios for both modes
✅ Understanding of physical interpretation

**Next: Activity 3 - Plotting responses and frequency response functions**


# Activity 3: Plot System Responses

## Objectives

1. **Unforced Response**: Plot the free vibration response to an initial displacement at the spring end
2. **Forced Response**: Plot the steady-state response for three different motor speeds
3. **Frequency Response Function (FRF)**: Plot the theoretical frequency response showing both modes

We'll use numerical values for visualization, which can later be replaced with actual experimental parameters.


In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.linalg import eig
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['lines.linewidth'] = 2

print("Libraries imported successfully!")


Libraries imported successfully!
